**<center><h1>Introduction</h1></center>**

Machine learning requires large quantities of data to train effective models. Some of this data can contain individuals' or companies’ sensitive information that must be secured to remain private. Security challenges surrounding machine learning are not just limited to data, and the setup of creating a secure work environment can be daunting. Azure Machine Learning (Azure ML) provides several mechanisms that offers granular control of the network environment, resources used within it, and the data being accessed. Proper measures can protect both from outside attackers and from internal issues like negligence or immature processes.

Many attacks on machine learning systems involve accessing models through insecure networks to steal or manipulate data to affect model performance and access sensitive data. By building better, more secure training data stores, locking down machine learning platforms, and controlling access to inputs and outputs, we can ensure data remains protected. These features are useful for data scientists, administrators, and operations engineers who want to create secure configurations compliant with their companies’ policies.


<mark>**Warning:** This module will give an understanding of the attack vectors to consider and the security tools available with Azure Machine Learning, but it is not a complete course in security. You should thoroughly read the Azure ML documentation and work with qualified cloud security experts before considering your architecture secure.</mark>

**<h2>Learning Objectives</h2>**

In this module, you will:

- Apply and understand Role-Based Access Control in Azure Machine Learning
- Describe how secrets are managed in Azure Machine Learning
- Use an Azure Machine Learning workspace with Azure Virtual Network

<hr>

**<center><h1>What is role-based access control?</h1></center>**

Azure Role-Based Access Control (Azure RBAC) is an authorization system that allows fine-grained access management of Azure Machine Learning resources. It enables you to manage team members' access to Azure cloud resources by assigning roles. These roles determine what assets team members can use in the workspace, and what they can do with those resources. This can be important if your team works with sensitive data such as hospital records that contain private medical information, or if you wish to restrict access to critical assets for junior team members within Azure Machine Learning. Using these roles, you can realistically reflect your organization's structure, ensuring responsibilities and critical assets are portioned to the correct individuals.



**<h2>User roles</h2>**


Azure RBAC roles can be assigned to individuals and groups. The rights assigned to each group are allocated as a permission-based system, with set access and restrictions being clearly defined. This control is applied at the workspace level and can only be changed by administrators or owners of the specific workspace within Azure Machine Learning. An Azure Machine Learning workspace, like other Azure resources, comes with three default roles when it is created. You can add users to the workspace and assign one of these roles:

- **Owners** have full access to the workspace, including the ability to view, create, edit, or delete assets in a workspace. Owners can also change role assignments.
- **Contributors** can view, create, edit, or delete assets in a workspace. For example, contributors can create an experiment, create or attach a compute cluster, submit a run, and deploy a web service.
- **Readers** can only perform read-only actions in the workspace. Readers can list and view assets, including datastore credentials in a workspace. Readers can't create or update these assets.

**<h3>Custom roles</h3>**

If the default roles do not meet your organization's need for more selective access control, you can create your own Custom roles. Custom roles give you the flexibility to develop permission-based rules for individuals' or groups that provide access while defining your own security stipulations to secure data or resources. You can make a role available at a specific workspace level, a specific resource group level, or a subscription level by defining the scope of your custom role, which we can see in the example JSON below.

Custom roles can be created by defining possible actions permitted and NotActions to restrict specific activities or access. You can create custom roles using Azure portal, Azure PowerShell, Azure CLI, or the REST API. Below we can see a custom role JSON request for a Data Scientist:
```
{
    "Name": "Data Scientist Custom",
    "IsCustom": true,
    "Description": "Can run experiment but can't create or delete compute or deploy production endpoints.",
    "Actions": [
        "Microsoft.MachineLearningServices/workspaces/*/read",
        "Microsoft.MachineLearningServices/workspaces/*/action",
        "Microsoft.MachineLearningServices/workspaces/*/delete",
        "Microsoft.MachineLearningServices/workspaces/*/write"
    ],
    "NotActions": [
        "Microsoft.MachineLearningServices/workspaces/delete",
        "Microsoft.MachineLearningServices/workspaces/write",
        "Microsoft.MachineLearningServices/workspaces/computes/*/write",
        "Microsoft.MachineLearningServices/workspaces/computes/*/delete", 
        "Microsoft.Authorization/*",
        "Microsoft.MachineLearningServices/workspaces/computes/listKeys/action",
        "Microsoft.MachineLearningServices/workspaces/listKeys/action",
        "Microsoft.MachineLearningServices/workspaces/services/aks/write",
        "Microsoft.MachineLearningServices/workspaces/services/aks/delete",
        "Microsoft.MachineLearningServices/workspaces/endpoints/pipelines/write"
    ],
    "AssignableScopes": [
        "/subscriptions/<subscription_id>/resourceGroups/<resource_group_name>/providers/Microsoft.MachineLearningServices/workspaces/<workspace_name>"
    ]
}
```
Note the Actions and NotActions above which define the permissions for the custom role and the assigned scope that is at the specific workspace level. In this example, the data scientist’s actions are defined through a wildcard (represented by the * sign) which extends a permission to everything that matches the action string you provide. So, in the above example the wildcard string adds all permissions related to any read, write, action, or deletion within the workspace.

However, if we look at the NotActions above, we can see restrictions on the deletion or creation of workspaces or new compute resources, amongst others. These restrictions take precedence over explicitly allowed actions.

**<h2>System</h2>**

A common challenge for developers is the management of secrets and credentials used to secure communication between different components. Azure Machine Learning relies on Azure Active Directory (Azure AD) for authentication and/or communication between other Azure cloud resources. Azure AD is a cloud-based identity and access management service that helps your employees' sign-in and access cloud resources on Azure.



**<h3>Authentication with Azure AD</h3>**

In general, there are three authentication workflows that you can use when connecting to the workspace:

**Interactive:** You use your account in Azure Active Directory to either manually authenticate or obtain an authentication token. Interactive authentication is used during **experimentation and iterative development**. It enables you to control access to resources (such as a web service) on a per-user basis.

**Service principal:** You create a service principal account in Azure Active Directory, and use it to authenticate or obtain an authentication token. A service principal is used when you need an automated process to authenticate to the service. For example, a continuous integration and deployment script that trains and tests a model every time the training code changes needs ongoing access and so would benefit from a service principal account.

**Azure CLI session:** You use an active Azure CLI session to authenticate. Azure CLI authentication is used during **experimentation and iterative development**, or when you need an **automated process to authenticate** to the service using a pre-authenticated session. You can log in to Azure via the Azure CLI on your local workstation, without storing credentials in code or prompting the user to authenticate.

**Managed identity:** When using the Azure Machine Learning SDK on an Azure Virtual Machine, you can use a managed identity for Azure. This workflow allows the VM to connect to the workspace using the managed identity, without storing credentials in code or prompting the user to authenticate. Azure Machine Learning compute clusters can also be configured to use a managed identity to access the workspace when training models.


**<h2>Managed Identities</h2>**

When configuring the Azure Machine Learning workspace in a trustworthy manner, it is vital to ensure that different cloud services associated with the workspace have the correct level of access. Managed identities allow you to authenticate services by providing an automatically managed identity for applications or services to use when connecting to Azure cloud services. Managed identities work with any service that supports Azure AD authentication, and provides activity logs so admins can see user activity such as log-in times, when operations were started, and by whom.

There are two types of managed identities:

- **System-assigned:** Some Azure services allow you to enable a managed identity directly on a service instance. When you enable a system-assigned managed identity, an identity is created in Azure AD tied to that service instance's lifecycle. By design, only that Azure resource can use this identity to request tokens from Azure AD, and when the resource is deleted, Azure automatically deletes the identity for you.
- **User-assigned:** You may also create a managed identity as a standalone Azure resource. You can create a user-assigned managed identity and assign it to one or more instances of an Azure service. The identity is managed separately from the resources that use it and will persist if a resource using it is removed. For simplicity, we recommend using system-assigned roles unless you require a custom access solution.

Once you have a managed identity, you can request tokens via a token endpoint on a resource such as a virtual machine. These tokens then work with users existing authorization from Azure RBAC to permit them to do actions, such as pull keys from Azure Key Vault or any other secret. Similarly, suppose that user has an Azure RBAC role that permits using an Azure storage solution. In that case, that token can be used to authenticate and either read or push data to storage without any credentials in their code. Operations on managed identities may be performed by using an Azure Resource Manager (ARM) template, the Azure portal, the Azure CLI, PowerShell, and REST APIs.

Below we can see the typical workflow for a managed identity within a virtual machine:

<img src="images/2-azure-identity.png" />


**<h2>Identities with compute clusters</h2>**

Azure Machine Learning compute clusters can use managed identities to authenticate access to Azure resources within Azure Machine Learning without including credentials in your code. This is useful to quickly provide the minimum required permissions to access resources while securing other critical resources. For example, during machine learning workflow, the workspace needs access to Azure Container Registry for Docker images, and storage accounts for training data. By default, the System-assigned identity is enabled directly on the Azure Machine Learning compute clusters and these resources will all be available to use. Once the compute cluster is deleted, Azure will automatically clean up the credentials and the identity in Azure Active Directory. Compute clusters can also support custom user-assigned identities assigned to multiple resources and will persist after resources are deleted.

During cluster creation or when editing compute cluster details, in the Advanced settings, toggle Assign a managed identity and specify a system-assigned identity or user-assigned identity. Note that Azure Machine Learning compute clusters support only one system-assigned identity or multiple user-assigned identities, not both concurrently.
<mark>**Note:** Managed identities can only be used with Azure Machine Learning when using the Azure Machine Learning SDK on an Azure Virtual Machine or Azure Machine Learning compute cluster. This workflow allows the virtual machine to connect to the workspace using the managed identity instead of the individual user's Azure AD account, and without storing credentials in code.</mark>





<hr>

**<center><h1>Exercise – Setup Azure Machine Learning with custom roles</h1></center>**

A [Machine Learning Workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is probably one of the most interesting resources to Data Scientists and ML Engineers.

Machine Learning Workspace is the top-level resource where you work with artifacts needed in Azure Machine Learning, such as datasets, models, and user roles.

In this exercise, we will set up an Azure Machine Learning Workspace, which will be used in following exercises and modules.

**<h2>Prerequisites</h2>**

- Azure Account and a Subscription

If you don't have an Azure subscription yet, please follow the instructions in the link below:

[Create a subscription in the Azure portal.](https://docs.microsoft.com/en-us/azure/cost-management-billing/manage/create-subscription#create-a-subscription-in-the-azure-portal)

**<h2>Sign in to Azure</h2>**

Sign in to the [Azure portal](https://portal.azure.com/).

**<h2>Create a Resource Group</h2>**

A Resource Group is where administrators can create logical groups of resources, like storage volumes, virtual machines and ML workspaces, so they can more easily track costs, apply access controls, and monitor activity.

Let's create a Resource Group for this exercise:

1. In the Azure portal, select **Resource Groups:**

<img src="images/resource-group-1.png" />

2. Select **New or Create** in the resource groups list:

<img src="images/resource-group-2.png" />

3. Fill in the form to add a new Resource Group

- Select your previously created subscription.
- Use **AI-Resource** as the Resource group's name.
- Select your preferred region.
- Select **Review and create**, then select **Create**.

<img src="images/resource-group-3.png" />

**<h2>Create ML Workspace</h2>**

<img src="images/.png" />

To create an ML Workspace, follow these steps:

1. Select **Home** to go back to the home page

2. Select **Create a resource** in the Azure portal:

<img src="images/3-workspace-1.png" />

3. Select **See all**.

<img src="images/3-workspace-2.png" />

4. Scroll down until the **AI + Machine Learning** option is visible and select **create.**
<img src="images/3-workspace-3.png" />

5. Fill in the form, selecting your current subscription and the resource group we had previously created (**AI-Resource**). Use **ml-workspace** as the workspace name:

<img src="images/3-workspace-4.png" />

6. Select the **Review + create** button to load a review confirmation screen:

<img src="images/3-workspace-5.png" />

7. Select Create to start deploying your new workspace (this may take a few minutes).

<mark>**Important**</mark>

<mark>Notice that the deployment above created other resources:</mark>

- <mark>A storage account used to store files used by the Workspace as well as data for experiments and model training</mark>
- <mark>An Application Insights instance, used to monitor predictive services in the Workspace.</mark>
- <mark>An Azure Key Vault instance, used to manage secrets such as authentication keys and credentials used by the Workspace.</mark>
- <mark>A container registry, created as-needed to manage containers for deployed models.</mark>


**<h2>Create a Custom Role and Assign it to the Workspace</h2>**

1. Select **Home** to go back to the home page

2. Open **Azure Cloud Shell**

    Select the Cloud Shell Icon:

<img src="images/powershell-1.png" />

or use the direct URL in your browser: https://shell.azure.com.

3. Select **Bash** or **PowerShell** (we will use bash):

<img src="images/powershell-2.png" />

**<h2>Define a Custom Role:</h2>**

Using a text editor of your choice, create a file called `data_scientist_custom_role.json`, replacing the information below:


<img src="images/image5.png" />
```
{
  "Name": "Data Scientist Custom",
  "IsCustom": true,
  "Description": "Can run experiment but can't create or delete compute.",
  "Actions": ["*"],
  "NotActions": [
    "Microsoft.MachineLearningServices/workspaces/*/delete",
    "Microsoft.MachineLearningServices/workspaces/write",
    "Microsoft.MachineLearningServices/workspaces/computes/*/write",
    "Microsoft.MachineLearningServices/workspaces/computes/*/delete",
    "Microsoft.Authorization/*/write"
  ],
  "AssignableScopes": [
    "/subscriptions/<subscription_id>/resourceGroups/<resource_group_name>/providers/Microsoft.MachineLearningServices/workspaces/<workspace_name>"
  ]
}
```

2. Upload the role definition file:

 - Select the **upload/Download files** icon
 - Select  **Upload**
 - Select the `data_scientist_custom_role.json` file you created above:

<img src="images/upload.png" />

Create a role definition using the `az role definition create` CLI command (notice that the last argument is the file we just uploaded):
```
az role definition create --role-definition data_scientist_custom_role.json
```

This adds the "Data Scientist Custom" role, which is limited to running experiments in the ml-workspace.

To verify and list existing roles using the Azure portal, select the option **Access Control (IAM)** on your ml-workspace then select the **Roles** tab:

<img src="images/custom-role.png" />





**<h2>Summary</h2>**

Congratulations!

In this unit you've covered the following topics:

- Creating a Resource group
- Creating a Machine Learning Workspace
- Using the Cloud Shell to run commands and uploading files to your account
- Defining a Custom Role for Machine Learning with access to your workspace
**<h2>More resources</h2>**

To read more about Workspaces and Custom Roles visit:

- [What is an Azure Machine Learning workspace?](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace)
- [Manage Azure Resource Manager resource groups by using the Azure portal](https://docs.microsoft.com/en-us/azure/azure-resource-manager/management/manage-resource-groups-portal#what-is-a-resource-group)
- [Manage access to an Azure Machine Learning workspace](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-assign-roles)

<hr>

**<center><h1>Keys and secrets with Azure Key Vault</h1></center>**

Most networked applications need to work with secrets, such as database connection strings or passwords. When perform machine learning on Azure, we normally use secrets to access training data or look at results. Particularly when working with private data, it’s important to make sure the secrets are properly managed.

**<h2>Don’t store secrets in source code</h2>**

Storing secrets in source code is impractical and a security anti-pattern. This is for multiple reasons:

- Changing passwords means updating source code, which can mean rebuilding and re-publishing applications.
- Hard-coded secrets make it awkward to work with different environments, such as staging and production environments. This also increases the risk of inadvertent modification or destruction of production environment data during development.
- All people with access to the source code gain access to all secrets. This makes it near impossible to ensure that only senior team members have access to sensitive resources. It also means that any sharing, or leak, of your source code also provides outside parties with your security keys.
- Source control, such as git, will typically retain old passwords in history. This means future team members gain access to all historical passwords.

One of the best alternatives to storing secrets in source code is to make them available in the application environment. In this pattern, your application requests secrets from the environment and then uses these to connect to the requisite resources. The aforementioned drawbacks of storing secrets in source code are eliminated, so long as each environment has different secret values, such as different passwords to access certain resources.

**<h2>Azure Key Vault</h2>**

Azure Key Vault provides secure storage of generic secrets for applications in Azure-hosted environments. Any type of secret can be stored, so long as its value is no larger than 25kb and it can be read and returned as a string. Secrets are named, and their content type (such as password or certificate) can optionally be stored alongside the value to provide a hint that assists in its interpretation when retrieved.

Secrets stored in Azure Key Vault are encrypted, optionally at the hardware level. This is handled transparently, and requires no action from the user or the application requesting the secrets. They can also be temporarily disabled, and automatically activate or expire on a certain date.

**<h2>How Key Vault works with Azure Machine Learning</h2>**

When you create an Azure Machine Learning workspace, this automatically creates a Key Vault. To view the Azure Key Vault associated with your workspace, open the workspace’s Overview tab. Your key vault appears on the right hand side.

<img src="images/resource-workspace.png" />

When you first create your workspace, your Key Vault will be automatically created. The Key Vault can be accessed through your application. For example, you can use the Azure Shell to set an environmental variable holding Key Store’s name, and save a password to that key store like so:
```
# Bash
# export the name of the vault to an environmental variable 
export KEY_VAULT_NAME=<your-unique-keyvault-name> 
 
# Save a new secret, called ExamplePassword 
az keyvault secret set --vault-name $KEY_VAULT_NAME --name "ExamplePassword" --value "hVFkk965BuUv"
```
This password is stored securely and is encrypted. As an example, a Python application using Azure Machine Learning’s SDK can access this key as follows:
```
# Python
''' 
Simple example of obtaining a secret from the keyvault. 
Assumes azure-identity and azure-keyvault-secrets have been 
pip installed 
''' 
 
import os 
from azure.keyvault.secrets import SecretClient 
from azure.identity import DefaultAzureCredential 
 
# Get the key vault name 
keyVaultName = os.environ["KEY_VAULT_NAME"] 
  
 
# Create a client to access the secret 
credential = DefaultAzureCredential() 
client = SecretClient(vault_url= f"https://{keyVaultName}.vault.azure.net", credential=credential) 
 
# Get a secret and print it to the console 
# Note that printing out passwords is bad practice and only  
# performed here for learning purposes 
retrieved_secret = client.get_secret("ExamplePassword") 
print(f"Your secret is '{retrieved_secret.value}'")
```
The output is the original password that we stored:
```
# Bash
Your secret is 'hVFkk965BuUv'
```

**<h2>Working with remote runs</h2>**

The above provides a generic solution to using Key Vault. Typically, with Azure Machine Learning, you will be executing code through a remote run.

The standard flow for using secrets in this context is:

- Log in to Azure and connect to your workspace,
- Set a secret in Workspace Key Vault,
- Submit a remote run, then
- Within the remote run, get the secret from Key Vault and use it.

When using the Python SDK and a run, secrets can be easily accessed directly. This is because a submitted run is aware of its workspace. For example:
```
# Python
# Code in submitted run 
from azureml.core import Experiment, Run 
 
run = Run.get_context() 
secret_value = run.get_secret(name=" ExamplePassword")
```
<img src="images/4-key-vault.png"/>

**<h2>Read more</h2>**

For more information about using secrets to secure and access your data, we recommend these resources:

- [Azure Key Vault](https://docs.microsoft.com/en-us/azure/key-vault/)
- [The Keyvault class in the Python SDK](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.keyvault.keyvault?view=azure-ml-py&viewFallbackFrom=azure-ml-py%22&preserve-view=true)
- [Using Secrets In Runs with Python](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-secrets-in-runs)
<hr>

**<center><h1>Secure your Azure Machine Learning network</h1></center>**

Before you begin training models, it's essential to secure your Azure Machine Learning network from outside intrusion. Without first securing your network, you can leave your data and models exposed to potentially malicious actors and lead to data theft or attacks that could negatively change model behavior. These alterations can often be difficult to spot due to the often large nature of datasets or parameters influencing model behavior. We will begin by separating your model training from the wider net to its own virtual network to avoid these problems.

**<h2>Virtual networks and security groups</h2>**

To secure the Azure Machine Learning workspace and compute resources, we will use a virtual network (VNet). An Azure VNet is the fundamental building block for your private network in Azure. VNet enables Azure resources, such as Azure Blob Storage and Azure Container Registry, to securely communicate with each other, the internet, and on-premises networks. VNet is similar to a traditional network that you'd operate in your own data center, but brings with it additional benefits of Azure's infrastructure such as scale, availability, and isolation. With a VNet, you can enhance security between Azure resources and filter network traffic to ensure only trusted users have access to the network.

<img src="images/5-azure-vnet-1.png"/>
In the above image, we can see a typical structure for a Virtual Network (VNet) comprised of:

- **IP address space:** When creating a VNet, you must specify a custom private IP address space using public and private (RFC 1918) addresses.
- **Subnets:** Shown above as separate virtual machines (VMs), subnets enable you to segment the virtual network into one or more sub-networks and allocate a portion of the virtual network's address space to each subnet, enhancing security and performance.
- **Network interfaces (NIC)** are the interconnection between a VM and a virtual network (VNet). When you create a VM in the Azure portal, a network interface is automatically created for you.
- **Network security groups (NSG)** can contain multiple inbound and outbound security rules that enable you to filter traffic to and from resources by source and destination IP address, port, and protocol.
- Load balancers can be configured to efficiently handle inbound and outbound traffic to VMs and VNets, while also offering metrics to monitor the health of VMs.

**<h2>Paths into a VNet</h2>**

Integrating Azure services to an Azure virtual network enables private access to the service from virtual machines or compute resources in the virtual network. You can integrate Azure services in your virtual network with the following options:

- **Servic** endpoints provide the identity of your virtual network to the Azure service. Once you enable service endpoints in your virtual network, you can add a virtual network rule to secure the Azure service resources to your virtual network. Service endpoints use public IP addresses.
- **Private** endpoints are network interfaces that securely connect you to a service powered by Azure Private Link. Private endpoint uses a private IP address from your VNet, effectively bringing the Azure services into your VNet.

<img src="images/5-azure-vnet-2.png"/>

You can connect your on-premises computers and networks to a VNet through a virtual private network (VPN) in several ways. **A Point-to-site VPN** is a connection between a virtual network and a single computer in your network. The communication is sent through an encrypted tunnel over the internet. Each computer that wants to establish connectivity with a VNet must configure its connection, so it's best used if you only have a few users who need to connect to the VNet. This connection type is great if you're just getting started as it requires little or no changes to your existing network.

While a **Site-to-site VPN** can be established between your on-premises VPN device and an Azure VPN Gateway that's deployed in a virtual network. This connection type enables any on-premises resource that you authorize to access a virtual network. The communication between your on-premises VPN device and an Azure VPN gateway is sent through an encrypted tunnel over the internet.

**ExpressRoute** can be used instead of a VPN If you wish to speed up creating private connections to Azure services. The service allows you to create private connections between Microsoft data centers and infrastructure on your premises or in another facility. ExpressRoute connections are separate from the public internet and offer high security, reliability, and speeds with lower latency than typical connections over the internet. ExpressRoute has a range of pricing options depending on your estimated bandwidth requirements.

**<h2>Securing the workspace and resources</h2>**

To begin securing your network, you will need to connect to your workspace via a private endpoint (private IP), which we will cover in the next exercise. The private endpoint can be added to a workspace through the Azure Machine Learning Python SDK, Azure CLI, or within the Networking tab of the Azure portal. From there, you can then limit access to your workspace to only occur over the private IP addresses.

However, this alone will not ensure end-to-end security by itself, so make sure other Azure services you are communicating with are also behind the VNet. Since communication to the workspace is then set to be only allowed from the VNet, any development environments that use the workspace must be members of the VNet unless you have configured the network to allow public IP connections.

The following methods can be used to connect to the secure workspace:

- **Azure VPN gateway** - Connects on-premises networks to the VNet over a private connection. Connection is made over the public internet. There are two types of VPN gateways that you might use:

 - **Point-to-site:** Each client computer uses a VPN client to connect to the VNet.
 - **Site-to-site:** A VPN device connects the VNet to your on-premises network.
- **ExpressRoute** - Connects on-premises networks into the cloud over a private connection. Connection is made using a connectivity provider.

- **Azure Bastion** - In this scenario, you create an Azure Virtual Machine (sometimes called a jump box) inside the VNet. You then connect to the VM using Azure Bastion. Bastion allows you to connect to the VM using either an RDP or SSH session from your local web browser. You then use the jump box as your development environment. Since it is inside the VNet, it can directly access the workspace.

**<h2>Security groups and traffic</h2>**

You can use an Azure network security group (NSG) to filter network traffic to and from Azure resources in an Azure virtual network. A network security group contains security rules that allow or deny inbound network traffic to, or outbound network traffic from, several types of Azure resources. NSGs are useful for controlling the traffic flow between VM subnets or limiting outbound communication by resources within an Azure VNet to the internet, which is enabled by default.

The following picture illustrates different scenarios for how network security groups might be deployed to allow network traffic to and from the internet over TCP port 80:

<img src="images/5-azure-vnet-3.png"/>

Application security groups (ASG) can also be used to configure network security as a natural extension of an application's structure, allowing you to group virtual machines and define network security policies based on those groups. You can reuse your security policy at scale without manual maintenance of explicit IP addresses. The platform handles the complexity of explicit IP addresses and multiple rule sets, simplifying the NSG rule definition process immensely.

**<h2>Service tags</h2>**

A service tag represents a group of IP address prefixes from a given Azure service. Microsoft manages the address prefixes encompassed by the service tag and automatically updates the service tag as addresses change, minimizing the complexity of frequent updates to network security rules.

You can use service tags in place of specific IP addresses when you create security rules to define network access controls on network security groups or Azure Firewall. By specifying the service tag name, such as ApiManagement, in the appropriate source or destination field of a rule, you can allow or deny the traffic for the corresponding service.

<img src="images/5-azure-vnet-4.png"/>

**<h2>Private endpoints & Private Link</h2>**

The Azure Machine Learning workspace can use Azure Private Link to create a private endpoint behind the VNet. Azure Private Link is a technology designed to provide connectivity to selected PaaS services. This provides a set of private IP addresses that can be used to access the workspace from within the VNet. Some of the services that Azure Machine Learning relies on can also use Azure Private Link, but some rely on network security groups or user-defined routing.

There are two key components of Azure Private Link:

- **Private endpoint** – a network interface connected to your virtual network, assigned with a private IP address. It is used to connect privately and securely to a service powered by Azure Private Link or a Private Link Service that you or a partner might own.
- **Private Link Service** – your own service, powered by Azure Private Link that runs behind an Azure Standard Load Balancer, enabled for Private Link access. This service can be privately connected with and consumed using Private Endpoints deployed in the user's virtual network.

When you create a private endpoint for your Azure resource, it provides secure connectivity between clients on your virtual network and your Azure resource. You can use private endpoints to communicate and ingress events directly from your virtual network to Azure resources securely over a private link without going through the public internet, boosting security. The private endpoint is assigned an IP address from the IP address range of your virtual network. The service is flexible, allowing connections between VNets with overlapping address spaces and connecting resources running in other regions, offering global reach.

<img src="images/5-azure-private-link.png"/>


















<hr>

**<center><h1>Exercise - Setup a secure Azure Virtual Network</h1></center>**

An [Azure Virtual Network (VNet)](https://docs.microsoft.com/en-us/azure/virtual-network/virtual-networks-overview) is similar to a traditional network but with the added benefits provided by Azure infrastructure.

VNets enable your resources, such as virtual machines (VMs) and ML workspaces, to securely communicate with each other and other networks or the internet.

A Machine Learning engineer can use them in a variety of scenarios, such as linking a VM to data stored on-premises, restricting access to a training API so that only personnel from their lab can see it, or exposing an inference endpoint to the internet.

In this exercise, we'll create a VNet and use it to secure access the workspace we created in the previous exercise. While this exercise gives an intuition about the mechanics of securing a workspace, this is not a complete solution. Refer to the documentation linked at the end of the exercise for complete instructions.

<mark>**Warning:** This exercise will only give a basic intuition as to the process involved in securing an ML workspace's network environment. For complete step-by-step process for a production environment, follow the link at the end of this exercise.</mark>

<mark>**Warning:** This exercise will block access to the affected workspace and should not be performed on a production workspace.</mark>


**<h2>Prerequisites</h2>**

- Basic knowledge of networking concepts.
- An Azure account and a subscription.
- A resource group Azure Machine Learning Workspace as created in the previous exercise.

**<h2>Sign in to Azure</h2>**

Sign in to the [Azure portal](https://portal.azure.com/?bundlingKind=DefaultPartitioner&configHash=2nQU4XU9OYiX&env=portal&helppanenewdesign=true&helppanevmproblemcards=true&l=en.en-us&pageVersion=9.337.0.179.220322-0014).

**<h2>Create a VNet</h2>**
Let's create a VNet for this exercise:

1. In the Azure portal, Select **Create a resource** in the upper left-hand corner of the portal.

2. In the search box, enter **Virtual Network**. Select **Virtual Network** in the search results.

3. In the **Virtual Network** page, select **Create**.

<img src="images/create-vnet.png"/>

4. In **Create virtual network**, enter or select this information in the **Basics** tab:


<img src="images/vnet-form-1.png"/>



<img src="images/image6.png"/>

5. Select the **IP Addresses** tab, or select the **Next: IP Addresses** button at the bottom of the page.

<mark>**Tip:** If your screen comes with the IPv4 address space and default subnet setup like in the image below, skip to step 9. </mark>


<img src="images/vnet-form-2.png"/>

6. In **IPv4 address space**, select the existing address space and change it to **10.1.0.0/16**.

7. Select + **Add subnet**, then enter **default** for **Subnet name** and **10.1.0.0/24** for **Subnet address range**.

8. Select **Add**.

9. Select the **Review + create** tab or select the **Review + create** button.

10. Select **Create** and wait a few moments for the deployment to finish:



<img src="images/vnet-deployment.png"/>

<mark>**Important:** For simplicity, we are creating a single subnet for our Virtual Network. A subnet is a range of IP addresses in the VNet. We could split a VNet into multiple subnets for organization and security, for example, a training subnet that only Data Scientists could use, and an inference subnet that is publicly available. </mark>

**<h2>Testing workspace access</h2>**

We just added a VNet to our resource group, but is it restricting access to our ml-workspace?

We can test that using [Azure Machine Learning Studio](https://docs.microsoft.com/en-us/azure/machine-learning/overview-what-is-machine-learning-studio) to access the contents of the workspace.

<mark>**Tip:** The Azure Machine Learning Workspace is a web portal with high-level tools for model training, deployment, and asset management.</mark>

1. Select **Home** to go back to the home page

2. Select the **ml-workspace** in the list of recent resources:

<img src="images/workspace-1.png"/>

3. Select the **Studio web URL** for your workspace:

<img src="images/workspace-2.png"/>

4. Azure Machine Learning Studio should open in a new tab or window. Scroll down until you can see the tabs below (Runs, Compute, Models, and Datasets):

<img src="images/workspace-no-warnings.png"/>

Each tab above represents a "folder" that stores the resources you and your team would use in Machine Learning.

Although you probably don't have any objects there yet, like in the image above, there should be no warnings or error messages, meaning that there's no network access restrictions on those resources yet.


**<h2>Secure network access to the ML Workspace</h2>**

So far we've created two independent resources:

- An ML workspace (**ml-workspace**)
- A VNet (**MLVNet**)

We now have to connect these two so that the workspace network traffic has to go through our VNet. In other words, we want our workspace available **only** to resources that are connected to the **MLVNet** virtual network.

To accomplish that, we need to define a [Private Endpoint](https://docs.microsoft.com/en-us/azure/private-link/private-endpoint-overview) for the ml-workspace resource.

<mark>**Tip:** A Private Endpoint is a network interface that uses a private IP Address from your VNet to create secure and private connections to a resource. </mark>

**<h2>Create a Private Endpoint</h2>**

1. Go to "Home", select **Resource Groups**, select the **MLResourceGroup** resource group, and then select your **ml_workspace** workspace:

<img src="images/ml-workspace-home.png"/>

2. In the left-hand menu, select **Private endpoint connections**:

<img src="images/private-endpoint-1.png"/>

3. Select **Private Endpoint** and fill the form with the following values:


<img src="images/private-endpoint-2.png"/>


<img src="images/image7.png"/>

4. Select the **Next: Resource** > button.

5. In the **Resource** tab, use the values below

<img src="images/private-endpoint-3.png"/>
<img src="images/image8.png"/>

6. Select the **Next: Configuration** > button.

Leave the suggested defaults:


<img src="images/private-endpoint-4.png"/>
<img src="images/image9.png"/>

7. Select **Review + Create** to validate the deployment, then select **Create** to deploy the endpoint (this can take a few moments):

<img src="images/private-endpoint-4 (1).png"/>

**<h2>Testing the network configuration</h2>**

We can make sure our workspace is inside the VNet now by testing if we still have access to its resources:

1. Reload the Azure Machine Learning Studio window (or [open the Studio](https://ml.azure.com/) again):

<img src="images/workspace-with-warnings.png"/>

2. As the warning shows, access to those resources is now blocked. That happens because your workspace is now inside the VNet, and it's configured to block all requests that don't originate from within the default subnet we created (remember we're trying to access it from the **outside** of the network perimeter).

**<h2>Secured but inaccessible</h2>**

You've just secured the network access to your ML workspace.

Notice that in the same way you can't access it, neither can any other resources in your organization that aren't part of the subnet.

Opening things up so that resources can connect requires some strategical planning!

We recommend reading our [how to secure a workspace using a vnet](https://azure/machine-learning/how-to-secure-workspace-vnet) guide for more in-depth reference.

**<h2>Summary</h2>**

In this unit you've covered the following topics:

- What a Virtual Network is and some of its uses
- Creating an Azure Virtual Network
- Creating subnets in a Virtual Network
- How to use Private Endpoints to secure network access to you Machine Learning workspaces
- Accessing a workspace using Azure Machine Learning Studio and check if resources are blocked

<hr>

**<center><h1>Knowledge check</h1></center>**

Choose the best response for each question. Then, select Check your answers.

Check your knowledge
1. What does creating Custom Roles allow you to do?

 - Custom roles allow you to customize what users can and can’t access in a workspace

 - Custom roles allow you to only perform read-only actions in a workspace

 - Custom roles allow you to customize the speaking voice used

2. What would you use the Azure Key Vault for?

 - To keep track of your house, work, and car keys

 - To hide your training models and data

 - To securely store and access secrets

3. Why use a virtual private network (VPN) for machine learning?

 - To assign roles

 - To secure your training models and data from malicious actors by encrypting your connection

 - To work offline

<hr>

**<center><h1>Summary</h1></center>**

In this module, you learned about some of the tools Azure Machine Learning provides to secure your data and training environment.

Now that you've viewed this module, you should be able to:

- Apply and understand Role-Based Access Control in Azure Machine Learning
- Describe how secrets are managed in Azure Machine Learning
- Put an Azure Machine Learning workspace in Azure Virtual Network


<hr>